<a href="https://colab.research.google.com/github/fralfaro/MAT306/blob/main/docs/labs/lab_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# MAT306 - Laboratorio N°03





**Objetivo**: Aplicar técnicas avanzadas de manipulación y análisis de datos con pandas sobre un conjunto real de datos de contenido de Netflix, reforzando buenas prácticas y métodos eficientes sin recurrir a `groupby`, `merge`, `pivot`, ni `join`.



**Dataset**:

Trabajaremos con el archivo `netflix_titles.csv`, que contiene información sobre los títulos disponibles en la plataforma Netflix hasta el año 2021.

| Variable       | Clase     | Descripción                                                                 |
|----------------|-----------|------------------------------------------------------------------------------|
| show_id        | caracter  | Identificador único del título en el catálogo de Netflix.                   |
| type           | caracter  | Tipo de contenido: 'Movie' o 'TV Show'.                                     |
| title          | caracter  | Título del contenido.                                                       |
| director       | caracter  | Nombre del director (puede ser nulo).                                       |
| cast           | caracter  | Lista de actores principales (puede ser nulo).                              |
| country        | caracter  | País o países donde se produjo el contenido.                                |
| date_added     | fecha     | Fecha en la que el título fue agregado al catálogo de Netflix.              |
| release_year   | entero    | Año de lanzamiento original del título.                                     |
| rating         | caracter  | Clasificación por edad (por ejemplo: 'PG-13', 'TV-MA').                      |
| duration       | caracter  | Duración del contenido (minutos o número de temporadas para series).        |
| listed_in      | caracter  | Categorías o géneros en los que está clasificado el contenido.              |
| description    | caracter  | Breve sinopsis del contenido.                                               |




In [2]:
import pandas as pd

# Cargar datos
df = pd.read_csv('https://raw.githubusercontent.com/fralfaro/MAT306/main/docs/labs/data/netflix_titles.csv')
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...



### Parte 1: Limpieza y preparación

1. Revisar y describir el dataset:

   * ¿Cuántas filas y columnas tiene?
   * ¿Qué tipos de datos hay?
   * ¿Cuántos valores nulos hay por columna?

2. Transformar la columna `date_added` a tipo fecha.

3. Crear columnas auxiliares con `assign`:

   * Año (`year_added`)
   * Mes (`month_added`)



In [14]:
print(df.shape)
df.info()
#df.describe() (estadística)
print(df.dtypes)
for col in df.select_dtypes("object"):
    print(col, df[col].map(type).value_counts(), "\n")
print(df["director"])
floats_in_director = [x for x in df["director"] if isinstance(x, float)]
print(floats_in_director)


(8807, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB
show_id         object
type            object
title           object
director        object
cast            object
country         object
date_added      object
release_year     int64
rating          object
duration        object
listed_in  

Revisar y describir el dataset:

¿Cuántas filas y columnas tiene?
El dataframe tiene 12 columnas y 8807 filas.

¿Qué tipos de datos hay?
Los datos son de tipo int64 y object. Más específicamente, los datos tipos object son strings y floats (notar que df.types lee muchos NaN como si fuesen floats).

In [15]:
#¿Cuántos valores nulos hay por columna?


print("\nValores nulos por columna")
print(df.isnull().sum())


Valores nulos por columna
show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64


In [16]:
#Transformar la columna date_added a tipo fecha.


df['date_added'] = pd.to_datetime(df['date_added'], errors='coerce')

#Crear columnas auxiliares con assign:

df = df.assign(
    year_added=df['date_added'].dt.year,
    month_added=df['date_added'].dt.month
)
display(df.head())

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,year_added,month_added
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",2021.0,9.0
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",2021.0,9.0
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,2021-09-24,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,2021.0,9.0
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,2021-09-24,2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",2021.0,9.0
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,2021.0,9.0


## Parte 2: Técnicas avanzadas de pandas

4. Utilizar `.loc` para seleccionar películas (`type == 'Movie'`) que fueron agregadas después del año 2018.

5. Utilizar `str.contains()` y `str.extract()`:

   * Filtrar títulos que contienen la palabra 'love' (sin distinguir mayúsculas/minúsculas).
   * Extraer la duración en minutos para las películas desde la columna `duration`.

6. Aplicar `explode()` sobre la columna `listed_in` para obtener una fila por cada género.

7. Obtener un top 10 de géneros más frecuentes utilizando `value_counts()`.

8. Aplicar `where()` y `mask()` para marcar las películas de más de 120 minutos como contenido largo en una nueva columna.

9. Utilizar `.loc` para filtrar películas que cumplen con:

   * Más de 100 minutos de duración.
   * Rating igual a `'R'`.
   * País igual a `'United States'`.

10. Utilizar `.style` para formatear visualmente el top 10 de películas más largas.

In [17]:
# 4. Utilizar .loc para seleccionar películas ('type == 'Movie'') que fueron agregadas después del año 2018.
movies_after_2018 = df.loc[(df['type'] == 'Movie') & (df['year_added'] > 2018)]
print("Películas agregadas después del 2018:")
display(movies_after_2018.head())

# 5. Utilizar str.contains() y str.extract():
#    * Filtrar títulos que contienen la palabra 'love' (sin distinguir mayúsculas/minúsculas).
titles_with_love = df[df['title'].str.contains('love', case=False, na=False)]
print("\nTítulos conteniendo 'love':")
display(titles_with_love.head())

#    * Extraer la duración en minutos para las películas desde la columna duration.
movies_df = df.loc[df['type'] == 'Movie'].copy()
movies_df['duration_minutes'] = movies_df['duration'].str.extract('(\d+)\s*min', expand=False).astype(float)
print("\nPelículas con duración en minutos:")
display(movies_df[['title', 'duration', 'duration_minutes']].head())


# 6. Aplicar explode() sobre la columna listed_in para obtener una fila por cada género.

df_exploded_genres = df.assign(listed_in=df['listed_in'].str.split(', ')).explode('listed_in')
print("\nDataFrame con géneros explotados:")
display(df_exploded_genres.head())

# 7. Obtener un top 10 de géneros más frecuentes utilizando value_counts().
top_10_genres = df_exploded_genres['listed_in'].value_counts().head(10)
print("\nTop 10:")
display(top_10_genres)

# 8. Aplicar where() y mask() para marcar las películas de más de 120 minutos como contenido largo en una nueva columna.

if 'duration_minutes' in movies_df.columns:
    movies_df['content_length'] = 'Short'
    movies_df['content_length'] = movies_df['content_length'].where(movies_df['duration_minutes'] <= 120, 'Long')
    # O usando mask:
    # movies_df['content_length'] = 'Long'
    # movies_df['content_length'] = movies_df['content_length'].mask(movies_df['duration_minutes'] <= 120, 'Short')

    display(movies_df[['title', 'duration_minutes', 'content_length']].head())
else:
    print("\nError")


# 9. Utilizar .loc para filtrar películas que cumplen con:
#    * Más de 100 minutos de duración.
#    * Rating igual a 'R'.
#    * País igual a 'United States'.

if 'duration_minutes' in movies_df.columns:
    filtered_movies = movies_df.loc[
        (movies_df['duration_minutes'] > 100) &
        (movies_df['rating'] == 'R') &
        (movies_df['country'] == 'United States')
    ]
    print("\nFiltered movies (duration > 100 min, Rating 'R', Country 'United States'):")
    display(filtered_movies.head())
else:
     print("\nError.")

# 10. Utilizar .style para formatear visualmente el top 10 de películas más largas.

if 'duration_minutes' in movies_df.columns:
    top_10_longest_movies = movies_df.sort_values(by='duration_minutes', ascending=False).head(10)
    print("\nTop 10 longest movies (styled):")
    display(top_10_longest_movies[['title', 'duration_minutes', 'duration']].style.background_gradient(subset=['duration_minutes'], cmap='Blues'))
else:
    print("\nError.")

Películas agregadas después del 2018:


<>:14: SyntaxWarning: invalid escape sequence '\d'
<>:14: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipython-input-1043472787.py:14: SyntaxWarning: invalid escape sequence '\d'
  movies_df['duration_minutes'] = movies_df['duration'].str.extract('(\d+)\s*min', expand=False).astype(float)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,year_added,month_added
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",2021.0,9.0
6,s7,Movie,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",NaN,2021-09-24,2021,PG,91 min,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...,2021.0,9.0
7,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...",2021-09-24,1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s...",2021.0,9.0
9,s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,2021-09-24,2021,PG-13,104 min,"Comedies, Dramas",A woman adjusting to life after a loss contend...,2021.0,9.0
12,s13,Movie,Je Suis Karl,Christian Schwochow,"Luna Wedler, Jannis Niewöhner, Milan Peschel, ...","Germany, Czech Republic",2021-09-23,2021,TV-MA,127 min,"Dramas, International Movies",After most of her family is murdered in a terr...,2021.0,9.0



Títulos conteniendo 'love':


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,year_added,month_added
25,s26,TV Show,Love on the Spectrum,NaN,Brooke Satchwell,Australia,2021-09-21,2021,TV-14,2 Seasons,"Docuseries, International TV Shows, Reality TV",Finding love can be hard for anyone. For young...,2021.0,9.0
158,s159,Movie,Love Don't Cost a Thing,Troy Byer,"Nick Cannon, Christina Milian, Kenan Thompson,...",United States,2021-09-01,2003,PG-13,101 min,"Comedies, Romantic Movies",A nerdy teen tries to make himself cool by ass...,2021.0,9.0
159,s160,Movie,Love in a Puff,Pang Ho-cheung,"Miriam Chin Wah Yeung, Shawn Yue, Singh Hartih...",Hong Kong,2021-09-01,2010,TV-MA,103 min,"Comedies, Dramas, International Movies",When the Hong Kong government enacts a ban on ...,2021.0,9.0
206,s207,Movie,"LSD: Love, Sex Aur Dhokha",Dibakar Banerjee,"Nushrat Bharucha, Anshuman Jha, Neha Chauhan, ...",India,2021-08-27,2010,TV-MA,112 min,"Dramas, Independent Movies, International Movies",This provocative drama examines how the voyeur...,2021.0,8.0
227,s228,Movie,Really Love,Angel Kristi Williams,"Kofi Siriboe, Yootha Wong-Loi-Sing, Michael Ea...",United States,2021-08-25,2020,TV-MA,95 min,"Dramas, Independent Movies, Romantic Movies",A rising Black painter tries to break into a c...,2021.0,8.0


...

,title,duration_minutes,content_length
0,Dick Johnson Is Dead,90.0,Short
6,My Little Pony: A New Generation,91.0,Short
7,Sankofa,125.0,Long
9,The Starling,104.0,Short
12,Je Suis Karl,127.0,Long



Filtered movies (duration > 100 min, Rating 'R', Country 'United States'):


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,year_added,month_added,duration_minutes,content_length
48,s49,Movie,Training Day,Antoine Fuqua,"Denzel Washington, Ethan Hawke, Scott Glenn, T...",United States,2021-09-16,2001,R,122 min,"Dramas, Thrillers",A rookie cop with one day to prove himself to ...,2021.0,9.0,122.0,Long
81,s82,Movie,Kate,Cedric Nicolas-Troyan,"Mary Elizabeth Winstead, Jun Kunimura, Woody H...",United States,2021-09-10,2021,R,106 min,Action & Adventure,"Slipped a fatal poison on her final job, a rut...",2021.0,9.0,106.0,Short
131,s132,Movie,Blade Runner: The Final Cut,Ridley Scott,"Harrison Ford, Rutger Hauer, Sean Young, Edwar...",United States,2021-09-01,1982,R,117 min,"Action & Adventure, Classic Movies, Cult Movies","In a smog-choked dystopian Los Angeles, blade ...",2021.0,9.0,117.0,Short
139,s140,Movie,Do the Right Thing,Spike Lee,"Danny Aiello, Ossie Davis, Ruby Dee, Richard E...",United States,2021-09-01,1989,R,120 min,"Classic Movies, Comedies, Dramas","On a sweltering day in Brooklyn, simmering rac...",2021.0,9.0,120.0,Short
144,s145,Movie,House Party,Reginald Hudlin,"Christopher Reid, Christopher Martin, Robin Ha...",United States,2021-09-01,1990,R,104 min,"Comedies, Cult Movies","Grounded by his strict father, Kid risks life ...",2021.0,9.0,104.0,Short



Top 10 longest movies (styled):


,title,duration_minutes,duration
4253,Black Mirror: Bandersnatch,312.000000,312 min
717,Headspace: Unwind Your Mind,273.000000,273 min
2491,The School of Mischief,253.000000,253 min
2487,No Longer kids,237.000000,237 min
2484,Lock Your Girls In,233.000000,233 min
2488,Raya and Sakina,230.000000,230 min
166,Once Upon a Time in America,229.000000,229 min
7932,Sangam,228.000000,228 min
1019,Lagaan,224.000000,224 min
4573,Jodhaa Akbar,214.000000,214 min




### Pregunta Desafío

11. ¿Cuáles son las combinaciones más frecuentes de género y rating en el dataset?
    (Sugerencia: utilizar `value_counts` con `subset=["genre", "rating"]` después de aplicar `explode()`).



### Bonus: Análisis de duplicados y limpieza

12. ¿Existen películas con el mismo nombre (`title`) pero con distinto año de lanzamiento (`release_year`)?
13. ¿Cuántos títulos únicos hay en total en la columna `title`?





In [18]:
# 11. ¿Cuáles son las combinaciones más frecuentes de género y rating en el dataset?

if 'df_exploded_genres' not in locals():
    df_exploded_genres = df.assign(listed_in=df['listed_in'].str.split(', ')).explode('listed_in')

genre_rating_combinations = df_exploded_genres.value_counts(subset=["listed_in", "rating"]).head(10)
print("Top 10:")
display(genre_rating_combinations)

# 12. ¿Existen películas con el mismo nombre (title) pero con distinto año de lanzamiento (release_year)?
duplicate_titles = df[df.duplicated(subset=['title', 'release_year'], keep=False)].sort_values(by='title')

if not duplicate_titles.empty:
    print("\nPeliculas con títulos repetidos:")

    title_release_counts = df.groupby('title')['release_year'].nunique()
    titles_with_multiple_release_years = title_release_counts[title_release_counts > 1].index.tolist()

    if titles_with_multiple_release_years:
        display(df[df['title'].isin(titles_with_multiple_release_years)].sort_values(by='title'))
    else:
         print("\nNo hay tales películas.")

else:
    print("\nNo hay tales películas.")


# 13. ¿Cuántos títulos únicos hay en total en la columna title?
unique_titles_count = df['title'].nunique()
print(f"\nTotal de títulos únicos: {unique_titles_count}")

Top 10:


listed_in               rating
International Movies    TV-MA     1130
                        TV-14     1065
Dramas                  TV-MA      830
International TV Shows  TV-MA      714
Dramas                  TV-14      693
International TV Shows  TV-14      472
Comedies                TV-14      465
TV Dramas               TV-MA      434
Comedies                TV-MA      431
Dramas                  R          375
Name: count, dtype: int64


No hay tales películas.

Total de títulos únicos: 8807
